In [ ]:
# 1. 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 환경 세팅

In [37]:
# 2. TPSMM 깃허브 클론 및 환경 준비
!git clone https://github.com/yoyo-nb/Thin-Plate-Spline-Motion-Model.git
%cd Thin-Plate-Spline-Motion-Model

!mkdir checkpoints
!pip install wldhx.yadisk-direct
!curl -L $(yadisk-direct https://disk.yandex.com/d/i08z-kCuDGLuYA) -o checkpoints/vox.pth.tar
!pip install imageio imageio_ffmpeg scikit-image

Cloning into 'Thin-Plate-Spline-Motion-Model'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 115 (delta 44), reused 30 (delta 30), pack-reused 46 (from 1)
Receiving objects: 100% (115/115), 32.65 MiB | 38.97 MiB/s, done.
Resolving deltas: 100% (51/51), done.
/content/Thin-Plate-Spline-Motion-Model/Thin-Plate-Spline-Motion-Model/Thin-Plate-Spline-Motion-Model
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  334M  100  334M    0     0  18.1M      0  0:00:18  0:00:18 --:--:-- 22.0M


In [38]:
# 3. 필요한 패키지 불러오기
import torch, os, json
import numpy as np
from glob import glob
from skimage import img_as_ubyte
from skimage.io import imread
from skimage.transform import resize
import imageio
from demo import load_checkpoints
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

In [39]:
# 4. 설정 정의
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset_name = 'vox' # ['vox', 'taichi', 'ted', 'mgif']
pixel = 256
if dataset_name == 'ted': # for ted, the resolution is 384*384
    pixel = 384

source_image_path = '/content/drive/MyDrive/KMY/dataset/test_img3.jpg'
# json_dir = '/content/drive/MyDrive/hanium_project/dataset/keypoints_json/gBR_sBM_cAll_d04_mBR0_ch01'
output_video_path = '/content/drive/MyDrive/KMY/dataset/generated_from_json.mp4'

config_path = 'config/vox-256.yaml'
checkpoint_path = 'checkpoints/vox.pth.tar'
predict_mode = 'relative'

In [40]:
import os
from glob import glob

# 전체 dataset 경로
base_dir = '/content/drive/MyDrive/KMY/dataset/keypoints_json'

# 하위 폴더 순회하며 모든 json 수집
all_json_paths = []

for subdir in sorted(os.listdir(base_dir)):
    sub_path = os.path.join(base_dir, subdir)
    if os.path.isdir(sub_path):
        jsons = sorted(glob(os.path.join(sub_path, '*.json')))
        all_json_paths.extend(jsons)

# 결과 확인
print(f"✔ 총 JSON 수: {len(all_json_paths)}")
print("예시 파일들:", all_json_paths[:5])


✔ 총 JSON 수: 13599
예시 파일들: ['/content/drive/MyDrive/KMY/dataset/keypoints_json/gBR_sBM_cAll_d04_mBR0_ch01/00000_keypoints.json', '/content/drive/MyDrive/KMY/dataset/keypoints_json/gBR_sBM_cAll_d04_mBR0_ch01/00001_keypoints.json', '/content/drive/MyDrive/KMY/dataset/keypoints_json/gBR_sBM_cAll_d04_mBR0_ch01/00002_keypoints.json', '/content/drive/MyDrive/KMY/dataset/keypoints_json/gBR_sBM_cAll_d04_mBR0_ch01/00003_keypoints.json', '/content/drive/MyDrive/KMY/dataset/keypoints_json/gBR_sBM_cAll_d04_mBR0_ch01/00004_keypoints.json']


#함수 정의


In [41]:
# # 5. JSON → TPSMM 키포인트 변환 함수 -> 키포인트 10개 씀
# def json_to_kp(json_path, img_size=256, device='cuda'):
#     """
#     OpenPose · MediaPipe style:
#     {"people": [{"pose_keypoints_2d": [x0,y0,conf0, x1,y1,conf1, ...]}]}
#     """
#     import json, torch, numpy as np

#     with open(json_path) as f:
#         data = json.load(f)

#     # ── ① 사람 선택 ──────────────────────────
#     if not data["people"]:
#         raise ValueError(f"No person found in {json_path}")
#     person = data["people"][0]                  # 여러 명일 땐 첫 번째 사람만 사용

#     # ── ② (x,y,conf) → (N,2) ndarray ────────
#     pts = np.array(person["pose_keypoints_2d"], dtype=np.float32)

#     # -- (x,y,conf) → (N,2) -----------------------------------------------
#     xy_all = pts.reshape(-1, 3)[:, :2]           # (N_all, 2)

#     # ----------------------------  ★ 핵심 ★  ------------------------------
#     # 모델이 학습한 kp_num = 10 에 맞춰 10개만 선택
#     SELECTED = [0, 5, 6, 7, 8, 9, 10, 11, 12, 13]
#     xy = xy_all[SELECTED]  # (10,2)



#     # ── ③ 픽셀 → [-1,1] 정규화 ───────────────
#     xy_norm = (xy / (img_size - 1)) * 2.0 - 1.0

#     kp      = torch.tensor(xy_norm, dtype=torch.float32,
#                            device=device).unsqueeze(0)   # (1,10,2)

#     # ── ④ Jacobian: 단위행렬 N개 ──────────────
#     jac = torch.eye(2, device=device).view(1,1,2,2).repeat(1, kp.shape[1], 1, 1)

#     # ── ④ fg_kp 키 추가 ─────────────────────
#     return {
#         # "value": kp_value,
#         # "jacobian": jac,
#         # "fg_kp": {
#         #     "value": kp_value,
#         #     "jacobian": jac
#         # }

#         "fg_kp": kp,           # <— Tensor
#         "fg_kp_jacobian": jac        # <— Tensor
#     }

In [42]:
NUM_TPS = 10               # 모델 고정
TOTAL_KP = NUM_TPS * 5     # 50 points

def json_to_kp(json_path, img_size=256, device='cuda'):
    import json, torch, numpy as np
    with open(json_path) as f:
        data = json.load(f)
    xy_all = np.array(data["people"][0]["pose_keypoints_2d"], dtype=np.float32)\
               .reshape(-1,3)[:, :2]      # (N_all,2)

    # ① 50개가 안 되면 마지막 포인트 반복
    if xy_all.shape[0] < TOTAL_KP:
        repeat = np.tile(xy_all[-1:], (TOTAL_KP - xy_all.shape[0], 1))
        xy = np.concatenate([xy_all, repeat], axis=0)[:TOTAL_KP]
    else:
        xy = xy_all[:TOTAL_KP]

    # ② [-1,1] 정규화
    xy_norm = (xy / (img_size-1))*2 - 1
    kp = torch.tensor(xy_norm, dtype=torch.float32, device=device).unsqueeze(0)  # (1,50,2)
    jac = torch.eye(2, device=device).view(1,1,2,2).repeat(1, kp.shape[1], 1, 1)
    return {"fg_kp": kp,
            "fg_kp_jacobian": jac,
            "xy:":xy }


In [43]:
# 6. JSON 시퀀스 불러오기
# kp_files = sorted(glob(os.path.join(json_dir, '*.json')))
kp_files = all_json_paths[:100]
kp_driving_seq = [json_to_kp(p, img_size=pixel, device=device) for p in kp_files]

# 7. 소스 이미지 불러오기
source_image = imread(source_image_path)
source_image = resize(source_image, (pixel, pixel))[..., :3]

# 8. TPSMM 모델 로드
inpainting, kp_detector, dense_motion_network, avd_network = load_checkpoints(
    config_path=config_path,
    checkpoint_path=checkpoint_path,
    device=device
)

#원본 메서드를 교체
import types
import torch.nn.functional as F

def create_deformed_source_image_dynamic(self, source_image, transformations):
    bs, _, h, w = source_image.shape
    num_trans = transformations.shape[1]            # ← ❶ 실제 개수 사용
    source_repeat = source_image.unsqueeze(1).unsqueeze(1)\
                                      .repeat(1, num_trans, 1, 1, 1, 1)\
                                      .view(bs * num_trans, -1, h, w)
    transformations = transformations.view(bs * num_trans, h, w, -1)
    deformed = F.grid_sample(source_repeat, transformations, align_corners=True)
    return deformed.view(bs, num_trans, -1, h, w)

dense_motion_network.create_deformed_source_image = types.MethodType(
    create_deformed_source_image_dynamic, dense_motion_network
)

In [44]:
# # 9. make_animation 수정 버전 (JSON 키포인트 사용)
# def make_animation_from_kp(source_image, kp_driving_seq,
#                            inpainting, dense_motion_network, avd_network,
#                            device='cuda', mode='relative'):
#     with torch.no_grad():
#         source = torch.tensor(source_image[np.newaxis].astype('float32').transpose(0,3,1,2)).to(device)
#         kp_source = kp_driving_seq[0]
#         predictions = []
#         for kp_driving in kp_driving_seq:
#             out = dense_motion_network(source, kp_source, kp_driving)

#             # 🔧 최종 optical flow인 'deformation'을 이용해 변형
#             deformation = out['deformation']  # (B, H, W, 2)
#             deformed = F.grid_sample(source, deformation, align_corners=True)

#             # 🔧 이제 그걸 인페인팅에 전달
#             inpainted = inpainting(deformed, out)
#             # inpainted = inpainting(out['deformed_source'],out)
#             predictions.append(inpainted.cpu().numpy().transpose(0,2,3,1)[0])
#     return predictions

In [45]:
# def make_animation_from_kp(source_image, kp_driving_seq,
#                            inpainting, dense_motion_network, avd_network,
#                            device='cuda', mode='relative'):
#     with torch.no_grad():
#         source = torch.tensor(source_image[np.newaxis].astype('float32').transpose(0,3,1,2)).to(device)
#         kp_source = kp_driving_seq[0]
#         predictions = []
#         for kp_driving in kp_driving_seq:
#             out = dense_motion_network(source, kp_source, kp_driving)
#             inpainted = inpainting(source, out)  # ✅ source and out (not just warped!)
#             predictions.append(inpainted.cpu().numpy().transpose(0,2,3,1)[0])
#     return predictions

In [46]:
def make_animation_from_kp(source_image, kp_driving_seq,
                           inpainting, dense_motion_network, avd_network,
                           device='cuda', mode='relative'):
    with torch.no_grad():
        source = torch.tensor(source_image[np.newaxis].astype('float32').transpose(0,3,1,2)).to(device)
        kp_source = kp_driving_seq[0]
        predictions = []
        for kp_driving in kp_driving_seq:
            out = dense_motion_network(source, kp_source, kp_driving)
            inpainted = inpainting(source, out)             # ✅ dict
            frame = inpainted['prediction']                 # ✅ extract tensor
            predictions.append(frame.cpu().numpy().transpose(0,2,3,1)[0])
    return predictions

In [47]:
# 마지막 sanity check
print(kp_driving_seq[0]['fg_kp'].shape)            # (1, 50, 2)
print(kp_driving_seq[0]['fg_kp_jacobian'].shape)   # (1, 50, 2, 2)
print(source_image.shape)                          # (256, 256, 3)

torch.Size([1, 50, 2])
torch.Size([1, 50, 2, 2])
(256, 256, 3)


#모델 구조 확인(테스트중)

In [ ]:
print([kp['fg_kp'].shape[1] for kp in kp_driving_seq])

[5, 5, 5, 5, 5, 5, 5, 5, 5]


In [ ]:
print(inpainting)

InpaintingNetwork(
  (first): SameBlock2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (norm): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
  )
  (down_blocks): ModuleList(
    (0): DownBlock2d(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (norm): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (pool): AvgPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0)
    )
    (1): DownBlock2d(
      (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (norm): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (pool): AvgPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0)
    )
    (2): DownBlock2d(
      (conv): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (norm): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stat

In [ ]:
print(ckpt.keys())

dict_keys(['inpainting_network', 'dense_motion_network', 'kp_detector', 'avd_network', 'bg_predictor'])


In [ ]:
# 키 이름 직접 확인
for k in dense_motion.keys():
    print(k)

down.weight
hourglass.encoder.down_blocks.0.conv.weight
hourglass.encoder.down_blocks.0.conv.bias
hourglass.encoder.down_blocks.0.norm.weight
hourglass.encoder.down_blocks.0.norm.bias
hourglass.encoder.down_blocks.1.conv.weight
hourglass.encoder.down_blocks.1.conv.bias
hourglass.encoder.down_blocks.1.norm.weight
hourglass.encoder.down_blocks.1.norm.bias
hourglass.encoder.down_blocks.2.conv.weight
hourglass.encoder.down_blocks.2.conv.bias
hourglass.encoder.down_blocks.2.norm.weight
hourglass.encoder.down_blocks.2.norm.bias
hourglass.encoder.down_blocks.3.conv.weight
hourglass.encoder.down_blocks.3.conv.bias
hourglass.encoder.down_blocks.3.norm.weight
hourglass.encoder.down_blocks.3.norm.bias
hourglass.encoder.down_blocks.4.conv.weight
hourglass.encoder.down_blocks.4.conv.bias
hourglass.encoder.down_blocks.4.norm.weight
hourglass.encoder.down_blocks.4.norm.bias
hourglass.decoder.up_blocks.0.conv.weight
hourglass.decoder.up_blocks.0.conv.bias
hourglass.decoder.up_blocks.0.norm.weight
hour

In [ ]:
import torch

ckpt = torch.load('checkpoints/vox.pth.tar', map_location='cpu')
dense_motion = ckpt['dense_motion_network']

for k, v in dense_motion.items():
    if 'hourglass.encoder.down_blocks.0.conv.weight' in k:
        in_channels = v.shape[1]
        num_tps = (in_channels - 4) // 8
        print(f"✅ Model was trained with num_tps = {num_tps}")
        print(f"🔍 Weight shape: {v.shape}")
        break


✅ Model was trained with num_tps = 10
🔍 Weight shape: torch.Size([128, 84, 3, 3])


#실행

In [48]:
# 10. 예측 수행
predictions = make_animation_from_kp(source_image, kp_driving_seq,
                                     inpainting, dense_motion_network, avd_network,
                                     device=device, mode=predict_mode)

# 11. 영상 저장
imageio.mimsave(output_video_path, [img_as_ubyte(frame) for frame in predictions], fps=30)

In [49]:
# 12. 영상 미리보기 (선택)
def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))
    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])
    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

HTML(display(source_image, [source_image]*len(predictions), predictions).to_html5_video())